In [1]:
!pip install --user --upgrade pip

In [2]:
! pip install numpy pandas plotly scipy matplotlib

In [3]:
! pip install numpy openpyxl

In [4]:
import numpy as np
import os
import pandas as pd
from datetime import datetime
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import tarfile
import zipfile
from scipy.io import loadmat
from urllib.request import urlretrieve
from os.path import isfile, isdir
import openpyxl
import plotly.express as px
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
%matplotlib inline

In [5]:
df = pd.read_excel("data/resultados_pruebas_saber_2020-1.xlsx")

In [7]:
df.describe(include='all', datetime_is_numeric=True)

,ESTU_TIPODOCUMENTO,ESTU_NACIONALIDAD,ESTU_GENERO,ESTU_FECHANACIMIENTO,ESTU_EXTERIOR,PERIODO,ESTU_CONSECUTIVO,ESTU_ESTUDIANTE,ESTU_PAIS_RESIDE,ESTU_DISC_FISICA,ESTU_DISC_SISTEM,ESTU_DISC_AUDITCASTELL,ESTU_DISC_AUDITLSC,ESTU_DISC_VISUALCEGUE,ESTU_DISC_VISUALBAJA,ESTU_DISC_VOZHABLA,ESTU_DISC_INTELEC,ESTU_DISC_ESPAUTISTA,ESTU_DISC_PSICOSOCIAL,ESTU_DISC_ASPERGER,ESTU_DEPTO_RESIDE,ESTU_COD_RESIDE_DEPTO,ESTU_MCPIO_RESIDE,ESTU_COD_RESIDE_MCPIO,ESTU_AREARESIDE,ESTU_ESTADOCIVIL,ESTU_COLE_TERMINO,ESTU_CODDANE_COLE_TERMINO,ESTU_COD_COLE_MCPIO_TERMINO,ESTU_OTROCOLE_TERMINO,ESTU_TITULOOBTENIDOBACHILLER,ESTU_PAGOMATRICULABECA,ESTU_PAGOMATRICULACREDITO,ESTU_PAGOMATRICULAPADRES,ESTU_PAGOMATRICULAPROPIO,ESTU_COMOCAPACITOEXAMENSB11,ESTU_TIPODOCUMENTOSB11,ESTU_SEMESTRECURSA,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,FAMI_TRABAJOLABORPADRE,FAMI_TRABAJOLABORMADRE,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_TIENESERVICIOTV,FAMI_TIENECOMPUTADOR,FAMI_TIENELAVADORA,FAMI_TIENEHORNOMICROOGAS,FAMI_TIENEAUTOMOVIL,FAMI_TIENEMOTOCICLETA,FAMI_TIENECONSOLAVIDEOJUEGOS,FAMI_CUANTOSCOMPARTEBAÑO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,INST_COD_INSTITUCION,INST_NOMBRE_INSTITUCION,ESTU_PRGM_ACADEMICO,ESTU_SNIES_PRGMACADEMICO,GRUPOREFERENCIA,ESTU_PRGM_CODMUNICIPIO,ESTU_PRGM_MUNICIPIO,ESTU_PRGM_DEPARTAMENTO,ESTU_NIVEL_PRGM_ACADEMICO,ESTU_METODO_PRGM,ESTU_NUCLEO_PREGRADO,ESTU_INST_CODMUNICIPIO,ESTU_INST_MUNICIPIO,ESTU_INST_DEPARTAMENTO,INST_CARACTER_ACADEMICO,INST_ORIGEN,ESTU_PRIVADO_LIBERTAD,ESTU_COD_MCPIO_PRESENTACION,ESTU_MCPIO_PRESENTACION,ESTU_DEPTO_PRESENTACION,ESTU_COD_DEPTO_PRESENTACION,MOD_RAZONA_CUANTITAT_PUNT,MOD_RAZONA_CUANTITATIVO_PNAL,MOD_RAZONA_CUANTITATIVO_PNBC,MOD_LECTURA_CRITICA_PUNT,MOD_LECTURA_CRITICA_PNAL,MOD_LECTURA_CRITICA_PNBC,MOD_COMPETEN_CIUDADA_PUNT,MOD_COMPETEN_CIUDADA_PNAL,MOD_COMPETEN_CIUDADA_PNBC,MOD_INGLES_PUNT,MOD_INGLES_DESEM,MOD_INGLES_PNAL,MOD_INGLES_PNBC,MOD_COMUNI_ESCRITA_PUNT,MOD_COMUNI_ESCRITA_DESEM,MOD_COMUNI_ESCRITA_PNAL,MOD_COMUNI_ESCRITA_PNBC,PUNT_GLOBAL,PERCENTIL_GLOBAL,PERCENTIL_NBC,ESTU_ESTADOINVESTIGACION,ESTU_EDAD
count,68725,68725,68597,68652,68725,68725.0,68725,68725,68725,27,3,8,8,1,10,1,3,1,2,2,68688,68688.000000,68688,68688.00000,58499,58190,37609,3.760900e+04,37609.000000,11543,58465,58461,58461,58463,58461,58475,57799,58465.0,65210,65100,56760,60492,64573,64460,64325,64056,64006,63846,63817,63840,63783,63863,63706,63606,68725.000000,68725,68725,68661.000000,68725,68725.00000,68725,68725,68725,68725,68725,68725.000000,68725,68725,68725,68725,68725,68725.000000,68725,68725,68725.000000,68725.000000,68567.000000,68566.000000,68725.000000,68567.000000,68566.000000,68725.000000,68567.000000,68566.000000,68705.000000,52346,68567.000000,68566.000000,67851.000000,58108.000000,67696.000000,67695.000000,68725.000000,68567.000000,68566.000000,68725,68652.000000
unique,6,17,2,NaN,1,NaN,68725,1,17,1,1,1,1,1,1,1,1,1,1,1,34,NaN,913,NaN,2,4,7201,NaN,NaN,9655,3,2,2,2,2,3,5,12.0,12,12,13,13,7,2,2,2,2,2,2,2,2,6,7,5,NaN,184,982,NaN,20,NaN,136,32,2,3,50,NaN,51,25,5,6,1,NaN,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
top,CC,COLOMBIA,M,NaN,NO,NaN,EK202010002443,ESTUDIANTE,COLOMBIA,x,x,x,x,x,x,x,x,x,x,x,BOGOTÁ,NaN,BOGOTÁ D.C.,NaN,Cabecera Municipal,Soltero,INST EDUC CEFA,NaN,NaN,otro,Bachiller académico,No,No,No,No,Repasó por cuenta propia,TI,4.0,Primaria incompleta,Secundaria (Bachillerato) completa,Trabaja por cuenta propia (por ejemplo plomero...,"Trabaja en el hogar, no trabaja o estudia",Estrato 2,Si,Si,Si,Si,No,No,No,No,3 o 4,No pago semestre,Más de 30 horas,NaN,SERVICIO NACIONAL DE APRENDIZAJE-SENA,TECNOLOGÍA EN CONTABILIDAD Y FINANZAS,NaN,TECNOLÓGICO EN ADMINISTRACIÓN Y TURISMO,NaN,BOGOTÁ D.C.,BOGOTÁ,TECNOLOGÍA,PRESENCIAL,ADMINISTRACIÓN,NaN,BOGOTÁ D.C.,BOGOTÁ,INSTITUCIÓN TECNOLÓGICA,OFICIAL NACIONAL,N,NaN,BOGOTÁ D.C.,BOGOTÁ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PUBLICAR,NaN
freq,67218,68663,34798,NaN,68725,NaN,1,68725,68663,

# Transformación de los Datos

### Variable: ESTU_TIPODOCUMENTO

In [8]:
df[['ESTU_TIPODOCUMENTO','PUNT_GLOBAL']].groupby("ESTU_TIPODOCUMENTO").agg(["count", "mean"])

PUNT_GLOBAL            
                         count        mean
ESTU_TIPODOCUMENTO                        
CC                       67218   93.080708
CE                          63  100.857143
CR                         125   98.200000
PE                           4  121.250000
PEP                          2   99.000000
TI                        1313   98.118812

### Descartado

Se descarta la variable ESTU_TIPODOCUMENTO, porque la gran mayoria de los estudiantes tienen una cedula, por lo que no representa un impacto en el resultado del puntaje global

### Variable: ESTU_NACIONALIDAD

In [9]:
df[['ESTU_NACIONALIDAD','PUNT_GLOBAL']].groupby("ESTU_NACIONALIDAD").agg(["count", "mean"])

PUNT_GLOBAL            
                           count        mean
ESTU_NACIONALIDAD                           
ALEMANIA                       1  135.000000
ANTIGUA Y BARBUDA              1  108.000000
BOLIVIA                        2  107.500000
CANADÁ                         1   88.000000
CHILE                          1  103.000000
COLOMBIA                   68663   93.185471
ECUADOR                        2  118.000000
EL SALVADOR                    4  138.750000
ESPAÑA                         2   96.500000
ESTADOS UNIDOS                 2   82.500000
HAITI                          2   80.500000
MÉXICO                         2  113.000000
NICARAGUA                      2   81.500000
PANAMÁ                         8   89.625000
PERÚ                           2  109.500000
REPÚBLICA DOMINICANA           2  122.500000
VENEZUELA                     28  104.214286

### Descartado

Se descarta la variable ESTU_NACIONALIDAD, porque la gran mayoria de los estudiantes son de Colombia, por lo que no representa un impacto en el resultado del puntaje global.

### Variable: ESTU_GENERO

In [10]:
df[['ESTU_GENERO','PUNT_GLOBAL']].groupby("ESTU_GENERO").agg(["count", "mean"])

PUNT_GLOBAL           
                  count       mean
ESTU_GENERO                       
F                 33799  92.733335
M                 34798  93.646474

### Descartado

Se descarta la variable ESTU_GENERO, porque tanto hombres como mujeres sacaron casi la misma media de puntuación global en las pruebas saber, por lo tanto, no afecta al resultado final que sea hombre o mujer.

### Variable: ESTU_FECHANACIMIENTO

### Descartado

Se descarta la variable ESTU_FECHANACIMIENTO, par ser reemplazada por la variable ESTU_EDAD, haciendole la siguiente tranformación:

In [12]:
hoy = datetime.today()

df['ESTU_FECHANACIMIENTO'] = pd.to_datetime(df['ESTU_FECHANACIMIENTO'], format='%Y-%m-%d', errors='coerce')

df['ESTU_EDAD'] = (hoy - pd.to_datetime(df['ESTU_FECHANACIMIENTO'], format='%Y-%m-%d')).astype('<m8[Y]').fillna(0).apply(lambda x: int(x))

edad_media = int(df['ESTU_EDAD'].mean())
mask = (df['ESTU_EDAD'] < 10) | (df['ESTU_EDAD'] > 100)
df.loc[mask, 'ESTU_EDAD'] = edad_media

df.to_excel("data/resultados_pruebas_saber_2020-1.xlsx", index=False)

In [13]:
df[['ESTU_EDAD','PUNT_GLOBAL']].groupby("ESTU_EDAD").agg(["count", "mean"])

PUNT_GLOBAL            
                count        mean
ESTU_EDAD                        
12                  1   45.000000
13                  1  104.000000
18                  5  111.200000
19                 31  102.483871
20                502   99.031873
21               3533   96.538070
22               9728   94.166632
23               8343   92.551600
24               6329   91.887186
25               4861   91.821847
26               4147   92.448999
27               3584   92.482701
28               3174   92.816635
29               2706   93.344789
30               2385   93.919078
31               2086   94.234899
32               1971   94.010147
33               1796   93.944321
34               1597   93.815905
35               1371   93.574034
36               1224   93.472222
37               1113   92.513926
38               1029   93.078717
39                922   94.355748
40                865   92.094798
41                840   93.540476
42                672   93.720238
43                663   93.911011
44                480   93.014583
45                404   93.113861
46                376   92.026596
47                285   93.540351
48                270   91.155556
49                226   90.181416
50                198   88.540404
51                181   87.381215
52                140   91.142857
53                127   92.685039
54                 94   88.531915
55                108   90.777778
56                 61   85.065574
57                 56   84.750000
58                 56   91.107143
59                 49   87.632653
60                 32   86.781250
61                 25   94.080000
62                 23   88.043478
63                  9   81.000000
64                 16   86.000000
65                  8   94.250000
66                  3   93.333333
67                  5   78.400000
68                  3   67.333333
69                  4   75.000000
70                  4   98.750000
71                  2   72.500000
83                  1  115.000000

### Variable: ESTU_EXTERIOR

In [14]:
df[['ESTU_EXTERIOR','PUNT_GLOBAL']].groupby("ESTU_EXTERIOR").agg(["count", "mean"])

PUNT_GLOBAL           
                    count       mean
ESTU_EXTERIOR                       
NO                  68725  93.195213

### Descartado

Se descarta la variable ESTU_EXTERIOR, porque nadie estudio en el exterior, por lo tanto no afeca al puntaje global.

### Variable: PERIODO

In [15]:
df[['PERIODO','PUNT_GLOBAL']].groupby("PERIODO").agg(["count", "mean"])

PUNT_GLOBAL           
              count       mean
PERIODO                       
20201         68725  93.195213

### Descartado

Se descarta la variable PERIODO, porque todos realizaron la prueba en el mismo periodo por lo que no es relevante para el analisis.

### Variable: ESTU_CONSECUTIVO

In [16]:
df[['ESTU_CONSECUTIVO','PUNT_GLOBAL']].groupby("ESTU_CONSECUTIVO").agg(["count", "mean"])

PUNT_GLOBAL       
                       count   mean
ESTU_CONSECUTIVO                   
EK202010000002             1  105.0
EK202010000003             1   72.0
EK202010000005             1   99.0
EK202010000007             1   87.0
EK202010000008             1   79.0
...                      ...    ...
EK202010099801             1   83.0
EK202010099802             1   92.0
EK202010099803             1   85.0
EK202010099804             1   80.0
EK202010099805             1  109.0

[68725 rows x 2 columns]

### Descartado

Se descarta la variable ESTU_CONSECUTIVO, porque representa un dato unico que identifica al estudiante, por lo tanto no es relevante para el analisis.

### Variable: ESTU_ESTUDIANTE

In [17]:
df[['ESTU_ESTUDIANTE','PUNT_GLOBAL']].groupby("ESTU_ESTUDIANTE").agg(["count", "mean"])

PUNT_GLOBAL           
                      count       mean
ESTU_ESTUDIANTE                       
ESTUDIANTE            68725  93.195213

### Descartado

Se descarta la variable ESTU_ESTUDIANTE, porque todos son estudiantes, por lo tanto no es relevante para el analisis

### Variable: ESTU_PAIS_RESIDE

### No se Descarta

No se descarta y se transforman los datos para crear una nueva categoria, y agrupar ahi los que no pertenecen a Colombia, debido a que los demas paises no llegan ni a 30 personas, porque es a partir de 30 que los datos se estabilizan.

In [19]:
df['ESTU_PAIS_RESIDE'] = np.where(df['ESTU_PAIS_RESIDE'] == 'COLOMBIA', 'COLOMBIA', 'OTROS')

df.to_excel("data/resultados_pruebas_saber_2020-1.xlsx", index=False)                                                                                                                                                

In [20]:
df[['ESTU_PAIS_RESIDE','PUNT_GLOBAL']].groupby("ESTU_PAIS_RESIDE").agg(["count", "mean"])

PUNT_GLOBAL            
                       count        mean
ESTU_PAIS_RESIDE                        
COLOMBIA               68663   93.185471
OTROS                     62  103.983871

### Variables: ESTU_DISC_FISICA, ESTU_DISC_SISTEM, ESTU_DISC_AUDITCASTELL, ESTU_DISC_AUDITLSC, ESTU_DISC_VISUALCEGUE, ESTU_DISC_VISUALBAJA, ESTU_DISC_VOZHABLA, ESTU_DISC_INTELEC, ESTU_DISC_ESPAUTISTA, ESTU_DISC_PSICOSOCIAL, ESTU_DISC_ASPERGER

### Descartadas

Se descartan las variables ESTU_DISC_FISICA, ESTU_DISC_SISTEM, ESTU_DISC_AUDITCASTELL, ESTU_DISC_AUDITLSC, ESTU_DISC_VISUALCEGUE, ESTU_DISC_VISUALBAJA, ESTU_DISC_VOZHABLA, ESTU_DISC_INTELEC, ESTU_DISC_ESPAUTISTA, ESTU_DISC_PSICOSOCIAL, ESTU_DISC_ASPERGER, las cuales representan diferentes tipos de discapacidades, por la poca cantidad de personas con discapacidad, que no representan un impacto significativo para realizar el modelo

### Variable: ESTU_DEPTO_RESIDE

### No se Descarta

No se descarta y se transforman los datos para crear una nueva categoria, y agrupar en esa categoria los departamentos que tengan menos de 30 personas, porque es a partir de 30 que los datos se estabilizan.

In [22]:
depto_counts = df['ESTU_DEPTO_RESIDE'].value_counts()

otros_deptos = depto_counts[depto_counts < 30].index.tolist()

df.loc[df['ESTU_DEPTO_RESIDE'].isin(otros_deptos), 'ESTU_DEPTO_RESIDE'] = "Otros"

df.to_excel("data/resultados_pruebas_saber_2020-1.xlsx", index=False)

In [23]:
df[['ESTU_DEPTO_RESIDE','PUNT_GLOBAL']].groupby("ESTU_DEPTO_RESIDE").agg(["count", "mean"])

PUNT_GLOBAL           
                        count       mean
ESTU_DEPTO_RESIDE                       
ANTIOQUIA               10159  93.249040
ARAUCA                    102  87.637255
ATLANTICO                3032  89.537269
BOGOTÁ                  19894  98.118930
BOLIVAR                  3483  87.933391
BOYACA                   2078  90.835900
CALDAS                    952  91.475840
CAQUETA                   230  85.491304
CASANARE                  439  90.662870
CAUCA                    1069  87.526660
CESAR                    1085  85.330876
CHOCO                      94  77.085106
CORDOBA                   652  84.107362
CUNDINAMARCA             5649  94.648610
GUAVIARE                   78  85.564103
HUILA                    1618  87.360321
LA GUAJIRA                363  84.977961
MAGDALENA                 853  84.587339
META                     1230  89.861789
NARIÑO                    959  89.473410
NORTE SANTANDER          1532  89.840731
Otros                      58  85.034483
PUTUMAYO                  310  86.890323
QUINDIO                   891  92.003367
RISARALDA                1297  94.299923
SANTANDER                3210  94.942679
SUCRE                     369  86.455285
TOLIMA                   2558  88.546130
VALLE                    4444  92.921242

### Variable: ESTU_COD_RESIDE_DEPTO

In [24]:
df[['ESTU_COD_RESIDE_DEPTO','PUNT_GLOBAL']].groupby("ESTU_COD_RESIDE_DEPTO").agg(["count", "mean"])

PUNT_GLOBAL           
                            count       mean
ESTU_COD_RESIDE_DEPTO                       
5.0                         10159  93.249040
8.0                          3032  89.537269
11.0                        19894  98.118930
13.0                         3483  87.933391
15.0                         2078  90.835900
17.0                          952  91.475840
18.0                          230  85.491304
19.0                         1069  87.526660
20.0                         1085  85.330876
23.0                          652  84.107362
25.0                         5649  94.648610
27.0                           94  77.085106
41.0                         1618  87.360321
44.0                          363  84.977961
47.0                          853  84.587339
50.0                         1230  89.861789
52.0                          959  89.473410
54.0                         1532  89.840731
63.0                          891  92.003367
66.0                         1297  94.299923
68.0                         3210  94.942679
70.0                          369  86.455285
73.0                         2558  88.546130
76.0                         4444  92.921242
81.0                          102  87.637255
85.0                          439  90.662870
86.0                          310  86.890323
88.0                           19  90.000000
91.0                           12  84.166667
94.0                            4  77.750000
95.0                           78  85.564103
97.0                            2  60.000000
99.0                           16  83.562500
99999.0                         5  88.800000

### Descartado

Se descarta la variable ESTU_COD_RESIDE_DEPTO, porque representa el codigo numerico del departamento lo cual no agrega información relevante con respecto a la variable ESTU_DEPTO_RESIDE

### Variable: ESTU_MCPIO_RESIDE

### No se Descarta

No se descarta y se transforman los datos para crear una nueva categoria, y agrupar en esa categoria los municipios que tengan menos de 30 personas, porque es a partir de 30 que los datos se estabilizan.

In [26]:
mcpio_counts = df['ESTU_MCPIO_RESIDE'].value_counts()

otros_mcpios = mcpio_counts[mcpio_counts < 30].index.tolist()

df.loc[df['ESTU_MCPIO_RESIDE'].isin(otros_mcpios), 'ESTU_MCPIO_RESIDE'] = "OTROS"

df.to_excel("data/resultados_pruebas_saber_2020-1.xlsx", index=False)

In [27]:
df[['ESTU_MCPIO_RESIDE','PUNT_GLOBAL']].groupby("ESTU_MCPIO_RESIDE").agg(["count", "mean"])

PUNT_GLOBAL            
                                  count        mean
ESTU_MCPIO_RESIDE                                  
ACACÍAS                              49   86.938776
AGUACHICA                           170   83.588235
AGUAZUL                              39   91.615385
AGUSTÍN CODAZZI                      38   81.526316
AMAGÁ                                34   95.382353
APARTADÓ                            299   81.431438
ARAUCA                               73   86.520548
ARJONA                              117   83.042735
ARMENIA                             606   92.041254
ATRATO                               40   74.775000
BARANOA                              54   86.962963
BARBOSA                             107   88.719626
BARRANCABERMEJA                     506   89.881423
BARRANQUILLA                       1524   90.869423
BELLO                              1001   95.844156
BOGOTÁ D.C.                       19894   98.118930
BOYACÁ                               36   84.444444
BUCARAMANGA                        1035   97.826087
BUENAVENTURA                        161   85.751553
CAJICÁ                              232   93.465517
CALARCÁ                             112   90.705357
CALDAS                              184   95.135870
CALI                               2227   94.738213
CAMPOALEGRE                          58   82.655172
CANDELARIA                           96   86.635417
CAREPA                               90   82.900000
CARTAGENA DE INDIAS                2843   89.018291
CARTAGO                             186   92.150538
CAUCASIA                            255   83.431373
CERETÉ                               62   80.677419
CHAPARRAL                            48   86.020833
CHIGORODÓ                            82   85.329268
CHINCHINÁ                            56   94.017857
CHIQUINQUIRÁ                         38   91.342105
CHOCONTÁ                             30   87.900000
CHÍA                                411   97.192214
CIRCASIA                             30   92.933333
CIÉNAGA                             119   80.957983
COGUA                                51   97.372549
COPACABANA                          152   96.125000
COROZAL                              41   86.780488
COTA                                 54   90.351852
CÚCUTA                             1107   89.702800
DAGUA                                37   88.972973
DOSQUEBRADAS                        357   93.761905
DUITAMA                             283   93.452297
EL CARMEN DE VIBORAL                 96   95.385417
EL CERRITO                           67   88.910448
EL SANTUARIO                         30   90.266667
EL TAMBO                             32   86.562500
ENVIGADO                            290   99.710345
ESPINAL                             478   82.684100
FACATATIVÁ                          333   86.924925
FLANDES                              61   84.163934
FLORENCIA                           197   84.989848
FLORIDA                              68   88.661765
FLORIDABLANCA                       462   97.683983
FONSECA                              63   83.682540
FUNDACIÓN                            30   79.633333
FUNZA                               252   97.380952
FUSAGASUGÁ                          245   91.983673
GACHANCIPÁ                           44   95.136364
GALAPA                               53   87.641509
GARZÓN                              232   85.137931
GIRARDOT                            227   89.850220
GIRARDOTA                            82   96.439024
GIRÓN                               259   94.733591
GRANADA                             109   89.220183
GUACARÍ                              47   83.510638
GUADALAJARA DE BUGA                 236   90.542373
GUADUAS                              31   87.483871
GUAMO                                48   81.854167
GUARNE                              104   95.105769
IBAGUÉ                             1531   91.649902
IPIALES                 

### Variable: ESTU_COD_RESIDE_MCPIO

In [28]:
df[['ESTU_COD_RESIDE_MCPIO','PUNT_GLOBAL']].groupby("ESTU_COD_RESIDE_MCPIO").agg(["count", "mean"])

PUNT_GLOBAL           
                            count       mean
ESTU_COD_RESIDE_MCPIO                       
5001.0                       4617  94.823695
5002.0                         11  81.727273
5004.0                          2  98.500000
5021.0                          1  97.000000
5030.0                         34  95.382353
...                           ...        ...
95025.0                         3  78.000000
97001.0                         2  60.000000
99001.0                        15  85.133333
99524.0                         1  60.000000
99999.0                         5  88.800000

[974 rows x 2 columns]

### Descartado

Se descarta la variable ESTU_COD_RESIDE_MCPIO, porque representa el codigo numerico del municipio lo cual no agrega información relevante con respecto a la variable ESTU_MCPIO_RESIDE

### Variable: ESTU_AREARESIDE

In [29]:
df[['ESTU_AREARESIDE','PUNT_GLOBAL']].groupby("ESTU_AREARESIDE").agg(["count", "mean"])

PUNT_GLOBAL           
                         count       mean
ESTU_AREARESIDE                          
Area Rural                9426  88.828878
Cabecera Municipal       49073  94.448536

### No se Descarta

No se descarta porque es relevante analizar en que influyen las areas donde se reside en el nivel educativo, y no hay quen hacerle transformación de los datos

### Variable: ESTU_ESTADOCIVIL

In [30]:
df[['ESTU_ESTADOCIVIL','PUNT_GLOBAL']].groupby("ESTU_ESTADOCIVIL").agg(["count", "mean"])

PUNT_GLOBAL           
                         count       mean
ESTU_ESTADOCIVIL                         
Casado                    4739  94.446719
Separado y/o Viudo         589  93.550085
Soltero                  45727  93.733440
Unión libre               7135  91.431535

### No se Descarta

No se descarta porque es relevante analizar en que influyen el estado civil de una persona en cuanto a nivel educativo, disponibilidad, etc, y no hay quen hacerle transformación de los datos

### Variable: ESTU_COLE_TERMINO

In [31]:
df[['ESTU_COLE_TERMINO','PUNT_GLOBAL']].groupby("ESTU_COLE_TERMINO").agg(["count", "mean"])

PUNT_GLOBAL            
                                       count        mean
ESTU_COLE_TERMINO                                       
ABSALON TORRES CAMACHO                    16   89.000000
ACAD CENTRAL DE SISTEMAS ACSI             10   85.900000
ACAD LA SALLE SAN BENILDO                  4  102.250000
ACAD MILIT MARISCAL SUCRE                  2   72.000000
ACAD MILIT NAL HEROES GRANADINOS           3   85.333333
...                                      ...         ...
VENTIUN ANGELES (INST EDUC DIST)          11  101.090909
VILLA CORELCA                              2   77.500000
VILLA DE LOS ANDES                         3   85.000000
VILLA DEL PRADO                            1   84.000000
VILLALOSADA                                1  111.000000

[7201 rows x 2 columns]

### No se Descarta

No se descarta porque es relevante analizar en que influyen la institucion donde se graduaron, ya que influye directamente en el nivel educativo, y se le hace transformación de los datos para agrupar en OTROS, los que tuvieran menos de 30 estudiantes.

In [32]:
pandas_by_inst = df[['ESTU_COLE_TERMINO','PUNT_GLOBAL']].groupby("ESTU_COLE_TERMINO").agg(["count", "mean"])

pandas_by_inst_without_mean = df[['ESTU_COLE_TERMINO','PUNT_GLOBAL']].groupby("ESTU_COLE_TERMINO").count()

s_gr_30 = list(pandas_by_inst_without_mean.query("PUNT_GLOBAL > 30").index)

df["ESTU_COLE_TERMINO"] = df.ESTU_COLE_TERMINO.apply(lambda x: x if x in s_gr_30 else 'OTROS')

df.to_excel("data/resultados_pruebas_saber_2020-1.xlsx", index=False)

In [34]:
df[['ESTU_COLE_TERMINO','PUNT_GLOBAL']].groupby("ESTU_COLE_TERMINO").agg(["count", "mean"])

PUNT_GLOBAL            
                                                         count        mean
ESTU_COLE_TERMINO                                                         
ALDEMAR ROJAS PLAZAS                                        40   99.250000
ANTONIO NARIÑO                                              45   81.111111
BRAULIO GONZALEZ                                            32   91.000000
CARLOS LOZANO Y LOZANO                                      46   93.565217
CARLOS PIZARRO LEON GOMEZ (INST EDUC DIST)                  33   95.787879
CASD HERMOGENES MAZA                                        33  101.757576
CEDID ALFONSO LOPEZ PUMAREJO                                37  101.486486
CEDID CIUDAD BOLIVAR                                        50   99.400000
CEDID GUILLERMO CANO ISAZA                                  34  103.882353
CEDID SAN PABLO BOSA                                        68   95.220588
CENT EDUC DIST ALMIRANTE PADILLA                            32   96.781250
CENT EDUC DIST CARLOS ALBAN HOLGUIN                         34  101.382353
CENT EDUC DIST EL PORVENIR                                  34  101.029412
CENT EDUC DIST GENERAL SANTANDER                            38  108.000000
CENT EDUC DIST GRAN COLOMBIANO                              36  103.138889
CENT EDUC DIST MARRUECOS Y MOLINOS                          32  100.093750
CENT EDUC DIST MIGUEL DE CERVANTES SAAVEDRA                 34  100.294118
CENT EDUC DIST NUEVO CHILE                                  34   98.647059
CIUDAD ESCOLAR DE COMFENALCO                                51   98.705882
COL CLARETIANO                                              43  103.581395
COL DIST EDUC BAS Y MEDIA ALBERTO LLERAS CAMARGO            36  101.416667
COL DIST EDUC BAS Y MEDIA LOS ALPES                         33   94.727273
COL DIST EDUC BAS Y MEDIA RODRIGO LARA BONILLA              58  100.758621
COL DIST ENRIQUE OLAYA HERRERA                              55  100.909091
COL DIST FERNANDO MAZUERA VILLEGAS                          84   93.964286
COL DIST JOSE FELIX RESTREPO                                44  102.886364
COL DIST LA MERCED                                          53  104.528302
COL DIST REP DE COSTA RICA                                  41   99.804878
COL DIST TOMAS RUEDA VARGAS                                 55  100.509091
COL DIST VENECIA                                            33   98.060606
COL INTEGD JUAN ATALAYA                                     34   92.176471
COL LUIS CARLOS GALAN SARMIENTO                             35   88.171429
COL NAL ANDRES BELLO                                        47  100.510638
COL NAL NICOLAS ESGUERRA                                    51  110.019608
COL NAL RESTREPO MILLAN                                     42  100.547619
COL PABLO CORREA LEON                                       40   93.250000
COL. MILITAR ALMIRANTE COLON                                78   97.525641
COLEGIO JOSÉ FRANCISCO SOCARRAS (IED) - SEDE PR...          31  102.354839
COLEGIO MANUEL CEPEDA VARGAS (IED) - SEDE PRINC...          33  104.090909
COLEGIO TECNICO INDUSTRIAL JOSE ELIAS PUYANA                39  101.717949
COLEGIO TECNICO MUNICIPAL SIMÓN BOLIVAR                     31   93.451613
COLEGIO TECNISISTEMAS - SEDE PRINCIPAL                      51   92.705882
COLEGIO UNIVERSITARIO                                       34   97.176471
ESCUELA NORMAL SUPERIOR                                     47   93.234043
I. E. SANTO TOMAS DE AQUINO                                 31   95.677419
I.E. ROBERTO VELANDIA - SEDE PRICIPAL                       33   99.787879
I.E. SAN ISIDORO - SEDE PRINCIPAL                           68   84.705882
I.E.M. LICEO INTEGRADO - SEDE PRINCIPAL                     34   92.176471
I.E.M. SANTIAGO PEREZ - SEDE PRINCIPAL                      39   97.641026
I.E.T. FELIX TIBERIO GUZMAN - SEDE PRINCIPAL                44   86.750000
I.E.T. GENERAL ROBERTO LEYVA SEDE CAMPESTRE - S...          37   82.054054
I

### Variable: ESTU_CODDANE_COLE_TERMINO

In [35]:
df[['ESTU_CODDANE_COLE_TERMINO','PUNT_GLOBAL']].groupby("ESTU_CODDANE_COLE_TERMINO").agg(["count", "mean"])

PUNT_GLOBAL            
                                count        mean
ESTU_CODDANE_COLE_TERMINO                        
0.000000e+00                        9   89.222222
1.050010e+10                       10   90.300000
1.500010e+10                        3  110.333333
3.080010e+10                        1   88.000000
3.080010e+10                        1   77.000000
...                               ...         ...
6.254730e+11                        1   72.000000
8.180011e+11                        1   73.000000
8.180011e+11                        4   71.500000
8.230011e+11                        1   51.000000
8.470010e+11                        3  100.333333

[7650 rows x 2 columns]

### Descartado

Se descarta la variable ESTU_CODDANE_COLE_TERMINO, porque representa el codigo numerico del municipio lo cual no agrega información relevante con respecto a la variable ESTU_COLE_TERMINO

### Variable: ESTU_COD_COLE_MCPIO_TERMINO

In [36]:
df[['ESTU_COD_COLE_MCPIO_TERMINO','PUNT_GLOBAL']].groupby("ESTU_COD_COLE_MCPIO_TERMINO").agg(["count", "mean"])

PUNT_GLOBAL            
                                  count        mean
ESTU_COD_COLE_MCPIO_TERMINO                        
5001.0                             2340   96.696154
5002.0                               16  100.125000
5004.0                                1  103.000000
5030.0                               25   95.840000
5031.0                               26   89.961538
...                                 ...         ...
97001.0                               5   80.200000
99001.0                               9   85.888889
99524.0                               6   73.166667
99624.0                               1   79.000000
99773.0                               1  108.000000

[1031 rows x 2 columns]

### Descartado

Se descarta la variable ESTU_COD_COLE_MCPIO_TERMINO, porque ya tenemos una variable para el municipio y otra para el colegio en el que se graduo, y no es relevente a que municipio pertenece cual colegio

### Variable: ESTU_OTROCOLE_TERMINO

In [37]:
df[['ESTU_OTROCOLE_TERMINO','PUNT_GLOBAL']].groupby("ESTU_OTROCOLE_TERMINO").agg(["count", "mean"])

PUNT_GLOBAL       
                                               count   mean
ESTU_OTROCOLE_TERMINO                                      
(I.E.D) PARAISO MIRADOR                            1  100.0
20 De julio                                        1  111.0
21 Angeles                                         1  113.0
A.A.E. MIRAVALLE                                   1   85.0
ACADEMIA AMERICANA DE SISTEMAS Y COMERCI           1   91.0
...                                              ...    ...
usminia D.C                                        1   99.0
validacion por icfes                               1   82.0
victor camargo alvarez                             1   82.0
vida y paz rodrigo arenas b                        1  126.0
ídem carlos perez mejia                            1   61.0

[9655 rows x 2 columns]

### Descartado

Se descarta la variable ESTU_OTROCOLE_TERMINO, porque todos los datos tienen maximo uno, por lo que no influye en el resultado del puntaje global

### Variable: ESTU_TITULOOBTENIDOBACHILLER

In [38]:
df[['ESTU_TITULOOBTENIDOBACHILLER','PUNT_GLOBAL']].groupby("ESTU_TITULOOBTENIDOBACHILLER").agg(["count", "mean"])

PUNT_GLOBAL           
                                        count       mean
ESTU_TITULOOBTENIDOBACHILLER                            
Bachiller académico                     39880  93.251204
Bachiller pedagógico o normalista         739  94.675237
Bachiller técnico                       17846  94.153424

### Descartado

Se descarta ESTU_TITULOOBTENIDOBACHILLER porque las media son practicamente las mismas, esto quiere decir que no representa una cambio considerable para el analisis

### Variable: ESTU_PAGOMATRICULABECA

In [39]:
df[['ESTU_PAGOMATRICULABECA','PUNT_GLOBAL']].groupby("ESTU_PAGOMATRICULABECA").agg(["count", "mean"])

PUNT_GLOBAL           
                             count       mean
ESTU_PAGOMATRICULABECA                       
No                           48219  93.521765
Si                           10242  93.653388

### Descartado

Se descarta ESTU_PAGOMATRICULABECA porque las media son practicamente las mismas, esto quiere decir que no representa una cambio considerable para el analisis

### Variable: ESTU_PAGOMATRICULACREDITO

In [40]:
df[['ESTU_PAGOMATRICULACREDITO','PUNT_GLOBAL']].groupby("ESTU_PAGOMATRICULACREDITO").agg(["count", "mean"])

PUNT_GLOBAL           
                                count       mean
ESTU_PAGOMATRICULACREDITO                       
No                              52463  93.341155
Si                               5998  95.328943

### Descartado

Se descarta ESTU_PAGOMATRICULACREDITO porque las media son practicamente las mismas, esto quiere decir que no representa una cambio considerable para el analisis

### Variable: ESTU_PAGOMATRICULAPADRES

In [41]:
df[['ESTU_PAGOMATRICULAPADRES','PUNT_GLOBAL']].groupby("ESTU_PAGOMATRICULAPADRES").agg(["count", "mean"])

PUNT_GLOBAL           
                               count       mean
ESTU_PAGOMATRICULAPADRES                       
No                             42734  93.584804
Si                             15729  93.438553

### Descartado

Se descarta ESTU_PAGOMATRICULAPADRES porque las media son practicamente las mismas, esto quiere decir que no representa una cambio considerable para el analisis

### Variable: ESTU_PAGOMATRICULAPROPIO

In [42]:
df[['ESTU_PAGOMATRICULAPROPIO','PUNT_GLOBAL']].groupby("ESTU_PAGOMATRICULAPROPIO").agg(["count", "mean"])

PUNT_GLOBAL           
                               count       mean
ESTU_PAGOMATRICULAPROPIO                       
No                             40894  92.738813
Si                             17567  95.423408

### Descartado

Se descarta ESTU_PAGOMATRICULAPROPIO porque las media son muy similares, esto quiere decir que no representa una cambio considerable para el analisis

### Variable: ESTU_COMOCAPACITOEXAMENSB11

In [43]:
df[['ESTU_COMOCAPACITOEXAMENSB11','PUNT_GLOBAL']].groupby("ESTU_COMOCAPACITOEXAMENSB11").agg(["count", "mean"])

PUNT_GLOBAL           
                                               count       mean
ESTU_COMOCAPACITOEXAMENSB11                                    
No realizó ninguna prueba de preparación       11050  95.082896
Repasó por cuenta propia                       42010  93.749893
Tomó un curso de preparación                    5415  88.815882

### No se descarta

No se descarta la variable ESTU_COMOCAPACITOEXAMENSB11

### Variable: ESTU_TIPODOCUMENTOSB11

In [44]:
df[['ESTU_TIPODOCUMENTOSB11','PUNT_GLOBAL']].groupby("ESTU_TIPODOCUMENTOSB11").agg(["count", "mean"])

PUNT_GLOBAL           
                             count       mean
ESTU_TIPODOCUMENTOSB11                       
CC                           21110  89.074751
CE                              68  99.661765
CR                             785  91.899363
PE                               9  93.666667
TI                           35827  96.200045

### No se Descarta

No se descarta ESTU_TIPODOCUMENTOSB11 porque los datos estan muy parejos, y esta relaciona con la variable ESTU_COMOCAPACITOEXAMENSB11

### Variable: ESTU_SEMESTRECURSA

In [45]:
df[['ESTU_SEMESTRECURSA','PUNT_GLOBAL']].groupby("ESTU_SEMESTRECURSA").agg(["count", "mean"])

PUNT_GLOBAL           
                         count       mean
ESTU_SEMESTRECURSA                       
1                         1055  85.646445
2                         2755  84.731397
3                         9520  93.048739
4                        22153  92.817587
5                         5428  97.302690
6                        10891  95.221743
7                         2897  97.080083
8                         1463  97.249487
9                         1007  96.677259
10                         542  94.616236
11                          70  94.742857
12 o más                   684  86.662281

### No se Descarta

No se descarta ESTU_SEMESTRECURSA porque es relevante para el analisis

### Variable: ESTU_SEMESTRECURSA

In [46]:
df[['FAMI_EDUCACIONPADRE','PUNT_GLOBAL']].groupby("FAMI_EDUCACIONPADRE").agg(["count", "mean"])

PUNT_GLOBAL            
                                           count        mean
FAMI_EDUCACIONPADRE                                         
Educación profesional completa              2947   99.205972
Educación profesional incompleta            1079  100.895273
Ninguno                                     2040   86.935294
No Aplica                                   1114   95.954219
No sabe                                     2449   98.349122
Postgrado                                    734  103.897820
Primaria completa                           7825   92.460319
Primaria incompleta                        16636   91.592811
Secundaria (Bachillerato) completa         13779   95.038827
Secundaria (Bachillerato) incompleta        8550   94.630058
Técnica o tecnológica completa              5904   97.220698
Técnica o tecnológica incompleta            2153   94.551324

### No se Descarta

No se descarta FAMI_EDUCACIONPADRE porque es relevante para el analisis

### Variable: FAMI_EDUCACIONMADRE

In [47]:
df[['FAMI_EDUCACIONMADRE','PUNT_GLOBAL']].groupby("FAMI_EDUCACIONMADRE").agg(["count", "mean"])

PUNT_GLOBAL            
                                           count        mean
FAMI_EDUCACIONMADRE                                         
Educación profesional completa              3148   99.110229
Educación profesional incompleta            1013   99.605133
Ninguno                                     1286   86.567652
No Aplica                                    147   88.965986
No sabe                                      383   94.091384
Postgrado                                    934  103.153105
Primaria completa                           8176   91.989114
Primaria incompleta                        13097   90.892571
Secundaria (Bachillerato) completa         16254   95.018826
Secundaria (Bachillerato) incompleta       10700   94.626542
Técnica o tecnológica completa              7638   97.661168
Técnica o tecnológica incompleta            2324   94.921687

### No se Descarta

No se descarta FAMI_EDUCACIONMADRE porque es relevante para el analisis

### Variable: FAMI_TRABAJOLABORPADRE

In [48]:
df[['FAMI_TRABAJOLABORPADRE','PUNT_GLOBAL']].groupby("FAMI_TRABAJOLABORPADRE").agg(["count", "mean"])

PUNT_GLOBAL            
                                                         count        mean
FAMI_TRABAJOLABORPADRE                                                    
Es agricultor, pesquero o jornalero                       8827   89.357993
Es dueño de un negocio grande, tiene un cargo d...         406   91.544335
Es dueño de un negocio pequeño (tiene pocos emp...        4902   94.570991
Es operario de máquinas o conduce vehículos (ta...        7607   96.120021
Es vendedor o trabaja en atención al público              3380   92.427811
No aplica                                                   80   85.662500
No sabe                                                   4680   95.404701
Pensionado                                                4462   96.638279
Tiene un trabajo de tipo auxiliar administrativ...        2035   94.392138
Trabaja como personal de limpieza, mantenimient...        5251   94.819273
Trabaja como profesional (por ejemplo médico, a...        2291  100.028808
Trabaja en el hogar, no trabaja o estudia                 2986   92.621567
Trabaja por cuenta propia (por ejemplo plomero,...        9853   94.982848

### No se Descarta

No se descarta FAMI_TRABAJOLABORPADRE porque es relevante para el analisis

### Variable: FAMI_TRABAJOLABORMADRE

In [49]:
df[['FAMI_TRABAJOLABORMADRE','PUNT_GLOBAL']].groupby("FAMI_TRABAJOLABORMADRE").agg(["count", "mean"])

PUNT_GLOBAL           
                                                         count       mean
FAMI_TRABAJOLABORMADRE                                                   
Es agricultor, pesquero o jornalero                       1784  89.151906
Es dueño de un negocio grande, tiene un cargo d...         238  88.798319
Es dueño de un negocio pequeño (tiene pocos emp...        4911  94.479536
Es operario de máquinas o conduce vehículos (ta...        1517  97.005933
Es vendedor o trabaja en atención al público              4292  93.767707
No aplica                                                   41  82.341463
No sabe                                                    697  89.355811
Pensionado                                                2381  98.265015
Tiene un trabajo de tipo auxiliar administrativ...        3462  97.110341
Trabaja como personal de limpieza, mantenimient...        6735  94.797476
Trabaja como profesional (por ejemplo médico, a...        2675  99.668037
Trabaja en el hogar, no trabaja o estudia                29324  93.553608
Trabaja por cuenta propia (por ejemplo plomero,...        2435  95.910472

### No se Descarta

No se descarta FAMI_TRABAJOLABORMADRE porque es relevante para el analisis

### Variable: FAMI_ESTRATOVIVIENDA

In [50]:
df[['FAMI_ESTRATOVIVIENDA','PUNT_GLOBAL']].groupby("FAMI_ESTRATOVIVIENDA").agg(["count", "mean"])

PUNT_GLOBAL            
                           count        mean
FAMI_ESTRATOVIVIENDA                        
Estrato 1                  16897   88.736699
Estrato 2                  29672   94.608419
Estrato 3                  15705   98.761350
Estrato 4                   1674  100.889486
Estrato 5                    320   98.140625
Estrato 6                     77   94.181818
Sin Estrato                  228   86.083333

### No se Descarta

No se descarta FAMI_ESTRATOVIVIENDA porque es relevante para el analisis

### Variable: FAMI_TIENEINTERNET

In [51]:
df[['FAMI_TIENEINTERNET','PUNT_GLOBAL']].groupby("FAMI_TIENEINTERNET").agg(["count", "mean"])

PUNT_GLOBAL           
                         count       mean
FAMI_TIENEINTERNET                       
No                        7981  87.806290
Si                       56479  95.141469

### No se Descarta

No se descarta FAMI_TIENEINTERNET porque es relevante para el analisis

### Descartado

Se descarta FAMI_TIENESERVICIOTV porque las medias estan muy similares, y no afecta realmente mucho que tenga TV o no.

### Variable: FAMI_TIENESERVICIOTV

In [52]:
df[['FAMI_TIENESERVICIOTV','PUNT_GLOBAL']].groupby("FAMI_TIENESERVICIOTV").agg(["count", "mean"])

PUNT_GLOBAL           
                           count       mean
FAMI_TIENESERVICIOTV                       
No                         17085  92.585192
Si                         47240  94.829382

### Descartado

Se descarta FAMI_TIENESERVICIOTV porque las medias estan muy similares, y no afecta realmente mucho que tenga TV o no.

### Variable: FAMI_TIENECOMPUTADOR

In [53]:
df[['FAMI_TIENECOMPUTADOR','PUNT_GLOBAL']].groupby("FAMI_TIENECOMPUTADOR").agg(["count", "mean"])

PUNT_GLOBAL           
                           count       mean
FAMI_TIENECOMPUTADOR                       
No                          8160  87.851348
Si                         55896  95.186936

### No se Descarta

No se descarta FAMI_TIENECOMPUTADOR porque es relevante para el analisis

### Variable: FAMI_TIENELAVADORA

In [54]:
df[['FAMI_TIENELAVADORA','PUNT_GLOBAL']].groupby("FAMI_TIENELAVADORA").agg(["count", "mean"])

PUNT_GLOBAL           
                         count       mean
FAMI_TIENELAVADORA                       
No                       12623  90.245900
Si                       51383  95.242824

### Descartado

Se descarta FAMI_TIENELAVADORA porque las medias estan muy similares, y no afecta realmente mucho que tenga Lavadora o no.

### Variable: FAMI_TIENEHORNOMICROOGAS

In [55]:
df[['FAMI_TIENEHORNOMICROOGAS','PUNT_GLOBAL']].groupby("FAMI_TIENEHORNOMICROOGAS").agg(["count", "mean"])

PUNT_GLOBAL           
                               count       mean
FAMI_TIENEHORNOMICROOGAS                       
No                             38286  92.427807
Si                             25560  97.061581

### Descartado

Se descarta FAMI_TIENEHORNOMICROOGAS porque las medias estan muy similares, y no afecta realmente mucho que tenga microhondas o no.

### Variable: FAMI_TIENEAUTOMOVIL

In [56]:
df[['FAMI_TIENEAUTOMOVIL','PUNT_GLOBAL']].groupby("FAMI_TIENEAUTOMOVIL").agg(["count", "mean"])

PUNT_GLOBAL           
                          count       mean
FAMI_TIENEAUTOMOVIL                       
No                        52603  93.690607
Si                        11214  97.000713

### Descartado

Se descarta FAMI_TIENEAUTOMOVIL porque las medias estan muy similares, y no afecta realmente mucho que tenga microhondas o no.

### Variable: FAMI_TIENEMOTOCICLETA

In [57]:
df[['FAMI_TIENEMOTOCICLETA','PUNT_GLOBAL']].groupby("FAMI_TIENEMOTOCICLETA").agg(["count", "mean"])

PUNT_GLOBAL           
                            count       mean
FAMI_TIENEMOTOCICLETA                       
No                          39694  95.139442
Si                          24146  92.807256

### Descartado

Se descarta FAMI_TIENEMOTOCICLETA porque las medias estan muy similares, y no afecta realmente mucho que tenga microhondas o no.

### Variable: FAMI_TIENECONSOLAVIDEOJUEGOS

In [58]:
df[['FAMI_TIENECONSOLAVIDEOJUEGOS','PUNT_GLOBAL']].groupby("FAMI_TIENECONSOLAVIDEOJUEGOS").agg(["count", "mean"])

PUNT_GLOBAL           
                                   count       mean
FAMI_TIENECONSOLAVIDEOJUEGOS                       
No                                 53110  93.492544
Si                                 10673  98.139324

### Descartado

Se descarta FAMI_TIENECONSOLAVIDEOJUEGOS porque las medias estan muy similares, y no afecta realmente mucho que tenga microhondas o no.

### Variable: FAMI_CUANTOSCOMPARTEBAÑO

In [59]:
df[['FAMI_CUANTOSCOMPARTEBAÑO','PUNT_GLOBAL']].groupby("FAMI_CUANTOSCOMPARTEBAÑO").agg(["count", "mean"])

PUNT_GLOBAL           
                               count       mean
FAMI_CUANTOSCOMPARTEBAÑO                       
1                               6278  97.270150
2                              12830  95.499454
3 o 4                          30518  94.064519
5 o 6                          10130  91.832379
MAS DE 6                        2396  90.335559
NINGUNA                         1711  96.023378

### Descartado

Se descarta FAMI_CUANTOSCOMPARTEBAÑO porque no es relevante para el analisis

### Variable: ESTU_VALORMATRICULAUNIVERSIDAD

In [60]:
df[['ESTU_VALORMATRICULAUNIVERSIDAD','PUNT_GLOBAL']].groupby("ESTU_VALORMATRICULAUNIVERSIDAD").agg(["count", "mean"])

PUNT_GLOBAL            
                                             count        mean
ESTU_VALORMATRICULAUNIVERSIDAD                                
Entre 3 y 5 millones                          1636  101.738386
Entre 5 y 7 millones                           199   90.175879
Entre 500 mil y un millon de pesos            5547   92.933658
Entre un millon y 3 millones de pesose       12861   97.443745
Mas de 7 millones                              131   87.519084
Menos de 500 mil pesos                        3192   95.236529
No pago semestre                             40140   93.036447

### No se Descarta

No se descarta ESTU_VALORMATRICULAUNIVERSIDAD porque es relevante para el analisis

### Variable: ESTU_HORASSEMANATRABAJA

In [61]:
df[['ESTU_HORASSEMANATRABAJA','PUNT_GLOBAL']].groupby("ESTU_HORASSEMANATRABAJA").agg(["count", "mean"])

PUNT_GLOBAL           
                              count       mean
ESTU_HORASSEMANATRABAJA                       
0                             11037  92.870526
Entre 11 y 20 horas            8032  92.615787
Entre 21 y 30 horas            6574  94.489048
Menos de 10 horas             11344  89.455395
Más de 30 horas               26619  97.227544

### No se Descarta

No se descarta ESTU_HORASSEMANATRABAJA porque es relevante para el analisis

In [9]:
df.drop(labels=['ESTU_TIPODOCUMENTO', 'ESTU_NACIONALIDAD', 'ESTU_GENERO', 'ESTU_FECHANACIMIENTO', 'ESTU_EXTERIOR', 'PERIODO', 'ESTU_CONSECUTIVO', 'ESTU_ESTUDIANTE', 'ESTU_DISC_FISICA', 'ESTU_DISC_SISTEM', 'ESTU_DISC_AUDITCASTELL', 'ESTU_DISC_AUDITLSC', 'ESTU_DISC_VISUALCEGUE', 'ESTU_DISC_VISUALBAJA', 'ESTU_DISC_VOZHABLA', 'ESTU_DISC_INTELEC', 'ESTU_DISC_ESPAUTISTA', 'ESTU_DISC_PSICOSOCIAL', 'ESTU_DISC_ASPERGER', 'ESTU_COD_RESIDE_DEPTO', 'ESTU_COD_RESIDE_MCPIO', 'ESTU_CODDANE_COLE_TERMINO', 'ESTU_COD_COLE_MCPIO_TERMINO', 'ESTU_OTROCOLE_TERMINO', 'ESTU_TITULOOBTENIDOBACHILLER', 'ESTU_PAGOMATRICULABECA', 'ESTU_PAGOMATRICULACREDITO'], axis=1, inplace=True)